<a href="https://colab.research.google.com/github/tomdu3/test/blob/main/test_ch_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHAPTER 12  
**Deploying Your Machine Learning Application**  

## Introduction  
In this chapter, we will explore how to transform your machine learning (ML) models from isolated experiments into fully functional, production-ready services by deploying them as RESTful application programming interfaces (APIs). As models progress beyond notebooks and scripts, the need to expose them reliably, often through Hypertext Transfer Protocol (HTTP) endpoints, becomes critical for real-world integration.  

To achieve this, we will introduce two of the most widely used Python web frameworks for serving ML models, Flask and FastAPI. You will learn how to build and test endpoints for real-time and batch inference, validate inputs, log predictions, and generate interactive documentation.  

Beyond basic deployment, we will also cover how to containerize your API with Docker, making it portable and scalable for use in cloud environments or microservice architectures. By the end of this chapter, you will not only understand how to expose your models as APIs, but also how to do this in the most secure and efficient way possible. This chapter will provide you with the practical skills and patterns needed to bridge the gap between model development and releasing your models to be production-grade inference services.  

## Structure  
In this chapter, we will cover the following topics:  
- Introducing Flask and FastAPI for building APIs  
- Creating RESTful endpoints for ML models  
- Introduction to Docker  
- Docker image creation, building and running containers  
- Best practice for deploying ML applications  

## Objectives  
By the end of this chapter, you will be able to deploy ML models as RESTful APIs using both Flask and FastAPI, enabling real-time and batch inference workflows. You will understand how to validate inputs, structure prediction responses, and automatically generate interactive API documentation with Swagger UI and ReDoc. You will also gain hands-on experience containerizing your application using Docker for consistent and portable deployment. Finally, you will learn key best practices for production deployment, including monitoring, logging, model versioning, and updating models without downtime.

---

## Introducing Flask and FastAPI for building APIs  
As ML models move from experimentation (in Jupyter notebooks or Python scripts) to production, one of the most common deployment patterns is exposing them as **inference APIs**. These are web endpoints that accept input data and return predictions. Python remains the dominant language in ML, and two of the most popular frameworks for serving models over HTTP are Flask and FastAPI.


### Flask, the simple and reliable option  
**Flask (https://flask.palletsprojects.com/en/stable/)** is a minimalist Python web framework that has long been a go-to choice for wrapping ML models in simple APIs. Its lightweight nature and low learning curve make it an attractive option for getting a model up and running quickly.  

The key features, for model serving, are captured in the following list:  
- Quick to set up with minimal boiler plating, which makes it ideal for prototyping  
- Easily integrates with any Python-based ML library (e.g., Scikit Learn, Tensorflow, and PyTorch)  
- Wide ecosystem for extensions like **request parsing** (extracts and formats incoming data), **logging** (records API activity for debugging and monitoring), and **Cross-Origin Resource sharing (CORS)** (allows safe requests from web apps on different domains)  
- Simple **synchronous request handling** (prevents the browser from executing additional code until the server responds), suitable for lightweight inference workloads  

The ideal case for Flask would be for wrapping traditional ML models or for internal tools, where latency and scalability are not critical concerns.

---

### FastAPI, high-performance framework for production inference  
**FastAPI (https://fastapi.tiangolo.com/)** is a modern Python web framework designed for high performance and developer productivity, especially useful for deploying ML models as APIs. It is built on **ASGI (Asynchronous Server Gateway Interface)**, a modern alternative to **WSGI (Web Server Gateway Interface)**. While WSGI handles one request at a time, which can become a bottleneck under heavy traffic, ASGI supports asynchronous programming, allowing **FastAPI** to handle many requests at once without blocking. This makes it highly efficient for real-time or high-throughput inference scenarios.  

**FastAPI** also leverages modern Python features like type annotations. Python is a dynamically typed language, meaning you do not have to specify variable types. However, with type annotations, you can add optional *hints* about what type of data a function expects or returns (e.g., int, str, List[float]). **FastAPI** uses these hints, along with libraries like **Pydantic (https://docs.pydantic.dev/latest/)**, to automatically validate inputs, serialize outputs, and generate clear, interactive documentation, all with minimal extra code.  

Together, these features make **FastAPI** a powerful choice for creating fast, scalable, and well-documented APIs, making it especially well-suited for serving ML models in production environments.  

Key features for model serving are as follows:  
- **Asynchronous request handling**: Ideal for input/output bound tasks such as loading large models or calling external APIs.  
- **Automatic input validation**: Using Pydantic reduces the chance of errors during inference.  
- **Built-in Swagger UI and ReDoc**: Built-in Swagger UI **(https://swagger.io/)** and ReDoc **(https://redocly.com/docs/redoc)** enable auto-generating API docs that make testing and debugging easier.  
- **High throughput and low latency**: Suitable for real-time (one request at a time) or batch inference (uploading data in batches) in production environments.  

**FastAPI** is ideal when serving deep learning models, running real-time predictions, or deploying at scale behind load balancers or in containerized environments like Docker and Kubernetes.  

When deploying ML models as APIs, **Flask** and **FastAPI** are both strong candidates. Flask offers a fast path to deployment for simple models and use cases, while FastAPI provides better performance, maintainability, and support for modern Python features, making it ideal for production-grade inference services. If you are experimenting or building proof-of-concept, Flask might be all you need, but if you are planning to scale, handle concurrent requests, or serve latency-sensitive models, FastAPI is the clear winner.

---

## Creating RESTful endpoints for ML models  
A RESTful endpoint is a URL through which clients (like web apps or scripts) can interact with a server using standard HTTP methods such as GET, POST, PUT, and DELETE (these will be explained ahead). When serving an ML model, you typically expose an endpoint, like `/predict`, where a client sends input data [`e.g., Java Script Object Notation (JSON)`] and the server responds with a prediction.  

REST stands for **Representational State Transfer**. In simple terms, a RESTful API organizes communication between systems around **resources** (like `/predict`) and **actions** (like sending a POST request). The main types of methods used in RESTful endpoints are explained hereunder:  
- **GET** is used to retrieve data from the server. We will use this in our endpoint to get a health status.  
- **POST** is the most used method for sending inference across HTTP. This says send the data to the server and create or process something. This is widely used on the internet for form submission when you fill in your credit card details on a website.  
- **PUT** is used to update or replace an existing resource, such as updating a user’s profile. This is used for managing model versioning or settings via an API.  
- **DELETE** is used to delete a resource on the server, such as deleting an account, record, or file. This is seldom used for inference endpoints.  

The common pattern for ML inference is to do the following:  
- A POST request is made to an endpoint like `/predict`.  
- The server processes the input, runs it through a **trained model** or **Sci-Kit Learn pipeline**, and returns the results as a **JSON** response.  

In the following sections, we will create an endpoint in Flask and FastAPI, and later we will dockerize these endpoints and make them microservices.

---

## Creating a RESTful Flask Endpoint

Now that you understand what RESTful endpoints are and why POST is typically used for model predictions, let's walk through building one in Flask. This section will show you how to structure your code, receive input data, run it through a trained model, and return the prediction.

In this section, we will use the model developed in **Chapter 10: Building Scikit-Learn Multi-Step Pipelines**, along with its corresponding preprocessor and serialized (saved) model. If you haven't read that chapter, don't worry. The supporting repository includes a `Model_Training` directory containing everything you need to retrain the model from scratch. In the steps that follow, we will set up all the necessary components to prepare for API integration.

### Folder Setup

When developing Flask and FastAPI applications, it's pivotal to separate these into sub-directories, as these directories will later be used to build Docker images. Please follow these steps to set up the initial folder structure:

1. Create a master directory and name it `flask`.
2. Inside this master directory, copy the `models` and `pipeline` directories from the `trained_model` folder, as these are our serialized artifacts which we'll need for our main Flask app.
3. When you've done this, your folder structure inside the `flask` root folder should just have the `models` and `pipeline` folders.

In the next section, we will create the Flask application.

---

## Creating the Flask application
At this point, we have the folder structure in place and are ready to create our `app.py` file inside the `flask` folder. It is not a Jupyter notebook this time, as we need to define it as a runnable script from inside the Docker container later on. We will build the script file step by step:

1. We will define our imports to make our script work. Alongside `flask`, we will be using `flask_restx` (as this simplifies the Swagger documentation process massively):

In [5]:
%pip install flask flask-restx loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.5 MB/s eta 0:00:00


In [6]:
import joblib
import numpy as np
import pandas as pd
from flask import Flask
from flask_restx import Api, Resource, fields
from loguru import logger
import os

2. It is good practice, when you are making production_grade applications, to have a logs directory to save our logs. We will be using **loguru (https://loguru.readthedocs.io/en/stable/)**, as this is easier than the default logging
inside the logging module. This makes a directory, and then we use **logger.add** to add a new config file and set the rotation to say overwrite the config when it gets 1 MB in size:

In [7]:
os.makedirs("logs", exist_ok=True)
logger.add("logs/api.log", rotation="1 MB")

1

3. Define the model path:

In [8]:
MODEL_PATH = "models/stack_class_pipe.joblib"

4. In the next step, we will define a try and exception block to try and load the model, if the model loads successfully, then the logger logs the success and if an exception occurs, then it is raised:

In [9]:
try:
  model = joblib.load(MODEL_PATH)
  logger.success(f"Model loaded successfully from {MODEL_PATH}")
except Exception as e:
  logger.error(f"Failed to load model from {MODEL_PATH}: {e}")
  raise

2025-08-04 10:32:22.059 | SUCCESS  | __main__:<cell line: 0>:3 - Model loaded successfully from models/stack_class_pipe.joblib


5. The next step will be to define the app, which creates a Flask application instance, using the current module’s name to set the app’s configuration and resource paths:

In [10]:
app = Flask(__name__)

6. In this step, we use a `flask_restx` API object linked to the Flask app, this provides versioning, a title and description for the API and enables Swagger documentation at the `/docs` endpoint:

In [11]:
api = Api(app, version="1.0", title="Customer Churn Prediction API",
          description="Predict churn risk category for customer",
          doc="/docs")

7. We now need a new namespace (endpoint) which will be our main predict method:

In [12]:
ns = api.namespace("predict",
                   description="Churn prediction operations")

8. Here, we define what the data structure of the dataset and trained model was, these fields need to match the values and types the model expects, but we know these values are there in the data:

In [13]:
input_model = api.model("Input", {
    'CustomerID': fields.String(example='CUST00001'),
    'Age': fields.Float(example=34.78),
    'Gender': fields.String(enum=['Male', 'Female']),
    'Tenure': fields.Float(example=22.46),
    'MonthlyCharges': fields.Float(example=86.31),
    'ServiceUsage': fields.Float(example=1.36),
    'ContractType': fields.String(enum=['Month-to-Month',
                                        'One-Year', 'Two-Year']),
    'PaymentMethod': fields.String(enum=['Credit Card', 'Bank Transfer',
                                         'Electronic Check', 'Mailed Check']),
    'CustomerSupportCalls': fields.Float(example=0.0)})

9. We need to define the meaning of each model output, as the raw outputs, 0, 1, or 2, are just numerical labels without context. To make them understandable, we will create a dictionary that maps each integer to a meaningful category:

In [14]:
CHURN_RISK_MAP = {
    0: "Low Risk",
    1: "Medium Risk",
    2: "High Risk"}

10. In this step, we create a REST API endpoint ("/") using `flask_restx`, which will accept POST requests for predicting a customer’s churn risk category. The Predict class, inheriting from Resource, defines this endpoint. When a request is received, it expects input data that matches the `input_model` schema. The data is extracted from the request payload and converted into a DataFrame, formatted for the model. The `CustomerID` is preserved for tracking, while the remaining features are used to generate a prediction and probability scores using a preloaded ML model.

- The model’s raw output, an integer representing a risk category, is then mapped to a human-readable label using the `CHURN_RISK_MAP` dictionary. The response includes the customer ID, the interpreted risk category, and the prediction probabilities for each class (**Low, Medium, High**). Logs are generated at each step for
traceability, and if an error occurs during processing, a 400-error response is returned
with the error message:

In [15]:
@ns.route("/")
class Predict(Resource):
    """
    Endpoint to predict churn risk category for a customer.
    """
    @ns.expect(input_model)
    def post(self):
        data = api.payload
        logger.info(f"Received prediction request: {data}")
        # Prepare the input DataFrame

        try:
            input_df = pd.DataFrame([{
                'CustomerID': data['CustomerID'],
                'Age': data['Age'],
                'Gender': data['Gender'],
                'Tenure': data['Tenure'],
                'MonthlyCharges': data['MonthlyCharges'],
                'ServiceUsage': data['ServiceUsage'],
                'ContractType': data['ContractType'],
                'PaymentMethod': data['PaymentMethod'],
                'CustomerSupportCalls': data['CustomerSupportCalls'],
            }])

            input_X = input_df.drop(columns=['CustomerID'])
            logger.debug(f"Input DataFrame for prediction: {input_X}")
            # Predict the churn risk category
            pred = model.predict(input_X)[0]
            pred_proba = model.predict_proba(input_X)[0]
            logger.info(f"Prediction result: {pred}, Probabilities: {pred_proba}")
            #Create a response dictionary
            response = {
                    "CustomerID": data['CustomerID'],
                    "prediction": CHURN_RISK_MAP.get(pred, "Unknown"),
                    "prediction_probs": {
                        "Low Risk": float(pred_proba[0]),
                        "Medium Risk": float(pred_proba[1]),
                        "High Risk": float(pred_proba[2])}
                        }
            logger.success(f"Prediction result for {data['CustomerID']}: {response}")
            return response, 200
        except Exception as e:
            logger.error(f"Error processing prediction request: {e}")
            return {"error": str(e)}, 400

11. For our API, we want to check the health of our endpoint, this is what this step does, as it sets up a `/health` GET endpoint using `flask_restx` to monitor the service’s readiness and functionality. When called, it logs the request with `logger.info()` and first checks whether the ML **model** is loaded into memory. If the model is `None`, it logs an error and returns a **500 response** indicating that the service is unhealthy. To further ensure the model is functional, it optionally performs a dummy inference using a hardcoded **DataFrame** containing representative input data. This simulates a prediction to confirm the model can process inputs without errors. If the inference fails, an error is logged and corresponding **500** error is returned. If all checks pass, the service responds with a **200** status and a message indicating the API is healthy:

In [16]:
@ns.route("/health")
class HealthCheck(Resource):
    def get(self):
        logger.info("Health check endpoint called")
        if model is None:
            logger.error("Model is not loaded!")
            return {"status": "unhealthy",
                    "reason": "Model not loaded"}, 500
        try:
            dummy = pd.DataFrame([{
                "Age": 30,
                "Gender": "Male",
                "Tenure": 5,
                "MonthlyCharges": 50,
                "ServiceUsage": 1,
                "ContractType": "Month-to-Month",
                "PaymentMethod": "Credit Card",
                "CustomerSupportCalls": 1,
            }])
            _ = model.predict(dummy)
        except Exception as e:
            logger.error(f"Model inference failed during health check: {e}")
            return {"status": "unhealthy", "reason": "Inference failed"}, 500
        logger.info("Health check passed")
        return {"status": "healthy"}, 200

In [17]:
if __name__ == "__main__":
    logger.info("Starting Churn Classifier API Server.")
    app.run(debug=False)

2025-08-04 10:32:53.904 | INFO     | __main__:<cell line: 0>:2 - Starting Churn Classifier API Server.


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [4]:
!unzip model_training.zip -d .

Archive:  model_training.zip
   creating: ./model_training/
   creating: ./model_training/data/
  inflating: ./model_training/data/synth_customer_churn.csv  
   creating: ./model_training/models/
  inflating: ./model_training/models/preprocessor.joblib  
  inflating: ./model_training/models/stack_class_pipe.joblib  
   creating: ./model_training/pipeline/
 extracting: ./model_training/pipeline/__init__.py  
  inflating: ./model_training/pipeline/binner.py  
  inflating: ./model_training/preprocess.py  
  inflating: ./model_training/train.py  
